In [ ]:
import re
import random
import math

# Load the graph from the file
my_file = open('/content/sample_data/10.clq', 'r')  # Replace with your file path
clique_line = my_file.readline()  # Read first line
node_edge_line = my_file.readline()  # Read second line

# Parse input
clique_size = int(re.findall(r'\b\d+\b', clique_line)[0])  # Extract max clique size
node = int(re.findall(r'\b\d+\b', node_edge_line)[0])  # Extract number of nodes

# Create adjacency matrix
adj_matrix = [[0] * node for _ in range(node)]
while True:
    next_line = my_file.readline().strip()
    if not next_line:
        break
    u, v = map(lambda x: int(x) - 1, re.findall(r'\b\d+\b', next_line))
    adj_matrix[u][v] = 1
    adj_matrix[v][u] = 1

my_file.close()

# Simulated Annealing Algorithm
def simulated_annealing(adj_matrix, max_iter, init_temp, end_temp, cooling_rate):
    n = len(adj_matrix)

    # Initialize with a random node
    current_clique = {random.randint(0, n - 1)}
    best_clique = current_clique.copy()
    current_temp = init_temp

    for iteration in range(max_iter):
        if current_temp <= end_temp:  # Break if temperature drops below the threshold
            break

        # Generate a neighbor by randomly adding or removing a node
        candidate = current_clique.copy()
        if random.random() < 0.5 and len(candidate) > 0:  # Remove a random node
            candidate.remove(random.choice(list(candidate)))
        else:  # Add a random connected node
            possible_nodes = [i for i in range(n) if i not in candidate and all(adj_matrix[i][v] for v in candidate)]
            if possible_nodes:
                candidate.add(random.choice(possible_nodes))

        # Check if the candidate is better or accept it probabilistically
        delta = len(candidate) - len(current_clique)
        if delta > 0 or math.exp(delta / current_temp) > random.random():
            current_clique = candidate

        # Update best solution
        if len(current_clique) > len(best_clique):
            best_clique = current_clique.copy()

        # Cool down the temperature
        current_temp *= cooling_rate

    return best_clique

# Parameters for Simulated Annealing
MAX_ITER = 10000  # Maximum number of iterations
INIT_TEMP = 100  # Initial temperature
END_TEMP = 0.001  # End temperature
COOLING_RATE = 0.995  # Cooling rate

# Run the algorithm
best_clique = simulated_annealing(adj_matrix, MAX_ITER, INIT_TEMP, END_TEMP, COOLING_RATE)

# Output
print(f"Max Clique Size (Benchmark): {clique_size}")
print(f"Max Clique Size Found: {len(best_clique)}")
print(f"Max Clique Nodes: {sorted(best_clique)}")